## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [3]:
import pandas as pd

In [51]:

# df=pd.read_csv('train.csv')
import pandas as pd

# Try specifying the encoding explicitly and skip bad lines
df = pd.read_csv('/content/train.csv', engine='python', on_bad_lines='skip')

df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [52]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [53]:
df.shape

(20822, 5)

In [9]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1.0
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0.0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1.0
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1.0
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1.0


In [54]:
df.tail()

,id,title,author,text,label
20817,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20818,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20819,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20820,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1
20821,20799,What Keeps the F-35 Alive,David Swanson,"David Swanson is an author, activist, journa...",1


In [10]:
df.isnull().sum()

,0
id,0
title,522
author,1858
text,54
label,16


In [11]:
###Drop Nan Values
df=df.dropna()


In [12]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1.0
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0.0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1.0
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1.0
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1.0


In [13]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [14]:
## Get the Dependent features
y=df['label']

In [15]:
X.shape

(17115, 4)

In [16]:
y.shape

(17115,)

In [17]:
import tensorflow as tf

In [18]:
tf.__version__

'2.17.0'

In [19]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [20]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [21]:
messages=X.copy()

In [22]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [23]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
19488,19473,Artificial intelligence ‘robot’ says Trump wil...,Admin,Other Writers \nAn Artificial Intelligence (AI...
19489,19474,Comment on Sweden Bans Christmas Street Lights...,jaded,"Posted on October 26, 2016 by Paul Joseph Wats..."
19490,19475,Arrests for Cannabis Possession Outnumber Arre...,The Mind Unleashed,Arrests for Cannabis Possession Outnumber Arre...
19491,19476,Clinton Vs. Trump: Latest Electoral Prediction...,beforeitsnews.com,(Before It's News)\nIt is fun to look at polls...


In [24]:
#messages.reset_index(inplace=True)

In [25]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
19488,19473,Artificial intelligence ‘robot’ says Trump wil...,Admin,Other Writers \nAn Artificial Intelligence (AI...
19489,19474,Comment on Sweden Bans Christmas Street Lights...,jaded,"Posted on October 26, 2016 by Paul Joseph Wats..."
19490,19475,Arrests for Cannabis Possession Outnumber Arre...,The Mind Unleashed,Arrests for Cannabis Possession Outnumber Arre...
19491,19476,Clinton Vs. Trump: Latest Electoral Prediction...,beforeitsnews.com,(Before It's News)\nIt is fun to look at polls...


In [26]:
import nltk
import re
from nltk.corpus import stopwords

In [27]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [30]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    # Use .iloc to access the element at the ith position
    review = re.sub('[^a-zA-Z]', ' ', messages['title'].iloc[i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [31]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [32]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [33]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[3562, 2959, 2441, 1549, 3463, 1030, 3456, 4060, 678, 4236],
 [4818, 1871, 345, 3862, 4318, 2454, 2632],
 [2998, 4541, 3380, 3634],
 [973, 865, 1608, 4801, 3563, 3715],
 [2489, 4318, 2575, 843, 4437, 2930, 4318, 4300, 3254, 2305],
 [3561,
  2297,
  3282,
  4386,
  1250,
  3139,
  2889,
  1480,
  1141,
  2983,
  1200,
  616,
  1861,
  3999,
  2632],
 [139, 4590, 3270, 4531, 4631, 576, 3552, 767, 4697, 1304, 3655],
 [1911, 1137, 4895, 1787, 3399, 571, 3139, 1662, 4697, 1304, 3655],
 [1067, 2622, 1910, 4516, 820, 2949, 3178, 827, 3139, 4092],
 [1035, 3621, 860, 2921, 4678, 1629, 1380, 3652],
 [3692, 4169, 971, 531, 4735, 4833, 2262, 2332, 2639, 2094, 4305],
 [4801, 3982, 3463, 2949, 3139, 3399],
 [3520, 155, 656, 2614, 1752, 3341, 1385, 1129, 882],
 [1090, 4014, 4755, 4709, 2875, 3996, 2982, 4697, 1304, 3655],
 [2779, 4803, 1776, 2176, 4326, 4697, 1304, 3655],
 [3557, 3686, 2681, 3516, 4380, 3199, 2143, 498, 3092, 1760],
 [1050, 1096, 1871],
 [591, 2139, 188, 1500, 3139, 940, 1937, 2632]

In [34]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [35]:
onehot_repr[1]

[4818, 1871, 345, 3862, 4318, 2454, 2632]

### Embedding Representation

In [36]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[3562 2959 2441 ...    0    0    0]
 [4818 1871  345 ...    0    0    0]
 [2998 4541 3380 ...    0    0    0]
 ...
 [2143 4036  627 ...    0    0    0]
 [ 345 3042 3139 ...    0    0    0]
 [3660 2949 1797 ...    0    0    0]]


In [37]:
embedded_docs[1]

array([4818, 1871,  345, 3862, 4318, 2454, 2632,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [38]:
embedded_docs[0]

array([3562, 2959, 2441, 1549, 3463, 1030, 3456, 4060,  678, 4236,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [39]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [40]:
len(embedded_docs),y.shape

(17115, (17115,))

In [41]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [42]:
X_final.shape,y_final.shape

((17115, 20), (17115,))

In [43]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [44]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 9s 34ms/step - accuracy: 0.7517 - loss: 0.4512 - val_accuracy: 0.9267 - val_loss: 0.1795
Epoch 2/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 11s 37ms/step - accuracy: 0.9509 - loss: 0.1300 - val_accuracy: 0.9265 - val_loss: 0.1943
Epoch 3/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 12s 45ms/step - accuracy: 0.9749 - loss: 0.0774 - val_accuracy: 0.9278 - val_loss: 0.2101
Epoch 4/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 10s 45ms/step - accuracy: 0.9827 - loss: 0.0548 - val_accuracy: 0.9251 - val_loss: 0.2676
Epoch 5/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - accuracy: 0.9901 - loss: 0.0308 - val_accuracy: 0.9205 - val_loss: 0.2921
Epoch 6/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 12s 44ms/step - accuracy: 0.9913 - loss: 0.0241 - val_accuracy: 0.9201 - val_loss: 0.3913
Epoch 7/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 11s 49ms/step - accuracy: 0.9879 - loss: 0.0327 - val_accuracy: 0.9131 - val_loss: 0.3628
Epoch 8/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 6s 32ms/step - accuracy: 0.9953 - loss: 0.0140 - val

### Adding Dropout

In [ ]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [45]:
y_pred=model.predict(X_test)

177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step


In [46]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [47]:
from sklearn.metrics import confusion_matrix

In [48]:
confusion_matrix(y_test,y_pred)

array([[2968,  242],
       [ 312, 2126]])

In [49]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9019121813031161

In [50]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       0.90      0.92      0.91      3210
         1.0       0.90      0.87      0.88      2438

    accuracy                           0.90      5648
   macro avg       0.90      0.90      0.90      5648
weighted avg       0.90      0.90      0.90      5648

